In [67]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [68]:
#Load api key from secret file
with open('/Data Enrichment/Week 2/API Calls/API-Calls/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [69]:
#Instantiate yelp api
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

## As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

In [70]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [71]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Redding_Thai-food.json"
JSON_FILE

'Data/results_in_progress_Redding_Thai-food.json'

In [72]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_Redding_Thai-food.json not found. Saving empty list to file.


In [73]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [74]:
Location = 'Redding, CA'
Term = 'Thai-food'

In [75]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location=Location,
                                       term=Term,
                                      offset=n_results)
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [76]:
## How many results total?
total_results = search_results['total']
total_results

18

In [77]:
## How many did we get the details for?
results_per_page = len(search_results['businesses'])
results_per_page

18

In [78]:
biz = pd.DataFrame(search_results['businesses'])
biz.head() 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,R_clG6jfCQCKHkEGpAUGxA,champa-garden-redding,Champa Garden,https://s3-media3.fl.yelpcdn.com/bphoto/6NUt98...,False,https://www.yelp.com/biz/champa-garden-redding...,889,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 40.59129, 'longitude': -122.35724}",[delivery],$$,"{'address1': '1107 Hilltop Dr', 'address2': ''...",+15302214999,(530) 221-4999,1942.462090
1,3ZgGOThmrvrYBnmKgyFYJg,thai-hut-redding,Thai Hut,https://s3-media3.fl.yelpcdn.com/bphoto/h9vbTV...,False,https://www.yelp.com/biz/thai-hut-redding?adju...,445,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 40.5634528076853, 'longitude': -1...",[delivery],$$,"{'address1': '1165 Hartnell Ave', 'address2': ...",+15302228405,(530) 222-8405,1549.345760
2,PrpQJCxSl9CDYWZcYTMwtg,janyas-thai-cuisine-redding,Janyas Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/PzQj2N...,False,https://www.yelp.com/biz/janyas-thai-cuisine-r...,278,"[{'alias': 'thai', 'title': 'Thai'}]",4.0,"{'latitude': 40.5979018677537, 'longitude': -1...",[delivery],$$,"{'address1': '630 N Market St', 'address2': ''...",+15302437682,(530) 243-7682,3386.564929
3,QtF_5NfBvn3PvRS9nKvHZw,rose-garden-redding-2,Rose Garden,https://s3-media4.fl.yelpcdn.com/bphoto/N7_n1R...,False,https://www.yelp.com/biz/rose-garden-redding-2...,181,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 40.5702422, 'longitude': -122.387...",[],$,"{'address1': '2825 S Market St', 'address2': '...",+15302438863,(530) 243-8863,2234.284231
4,H6OFH-qwu5tz3StMCDnIsw,thai-cafe-redding,Thai Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/IzAJU8...,False,https://www.yelp.com/biz/thai-cafe-redding?adj...,111,"[{'alias': 'thai', 'title': 'Thai'}]",4.0,"{'latitude': 40.5865325927734, 'longitude': -1...",[delivery],$$,"{'address1': '820 Butte St', 'address2': '', '...",+15302435523,(530) 243-5523,2251.315624


## Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

In [79]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((search_results['total']-n_results)/ results_per_page)
n_pages

1

In [80]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [81]:
#Install tqdm
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [82]:
# join new results with old list with extend and save to file
previous_results.extend(search_results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [83]:
#Import tqdm notebook
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/1 [00:00<?, ?it/s]

In [84]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(search_results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/1 [00:00<?, ?it/s]

In [85]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,R_clG6jfCQCKHkEGpAUGxA,champa-garden-redding,Champa Garden,https://s3-media3.fl.yelpcdn.com/bphoto/6NUt98...,False,https://www.yelp.com/biz/champa-garden-redding...,889,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 40.59129, 'longitude': -122.35724}",[delivery],$$,"{'address1': '1107 Hilltop Dr', 'address2': ''...",15302214999,(530) 221-4999,1942.462090
1,3ZgGOThmrvrYBnmKgyFYJg,thai-hut-redding,Thai Hut,https://s3-media3.fl.yelpcdn.com/bphoto/h9vbTV...,False,https://www.yelp.com/biz/thai-hut-redding?adju...,445,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 40.5634528076853, 'longitude': -1...",[delivery],$$,"{'address1': '1165 Hartnell Ave', 'address2': ...",15302228405,(530) 222-8405,1549.345760
2,PrpQJCxSl9CDYWZcYTMwtg,janyas-thai-cuisine-redding,Janyas Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/PzQj2N...,False,https://www.yelp.com/biz/janyas-thai-cuisine-r...,278,"[{'alias': 'thai', 'title': 'Thai'}]",4.0,"{'latitude': 40.5979018677537, 'longitude': -1...",[delivery],$$,"{'address1': '630 N Market St', 'address2': ''...",15302437682,(530) 243-7682,3386.564929
3,QtF_5NfBvn3PvRS9nKvHZw,rose-garden-redding-2,Rose Garden,https://s3-media4.fl.yelpcdn.com/bphoto/N7_n1R...,False,https://www.yelp.com/biz/rose-garden-redding-2...,181,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 40.5702422, 'longitude': -122.387...",[],$,"{'address1': '2825 S Market St', 'address2': '...",15302438863,(530) 243-8863,2234.284231
4,H6OFH-qwu5tz3StMCDnIsw,thai-cafe-redding,Thai Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/IzAJU8...,False,https://www.yelp.com/biz/thai-cafe-redding?adj...,111,"[{'alias': 'thai', 'title': 'Thai'}]",4.0,"{'latitude': 40.5865325927734, 'longitude': -1...",[delivery],$$,"{'address1': '820 Butte St', 'address2': '', '...",15302435523,(530) 243-5523,2251.315624


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
31,HSYAmm-BN1etAeWPrgZb_A,kanya-market-redding,Kanya Market,https://s3-media2.fl.yelpcdn.com/bphoto/0ruT9S...,False,https://www.yelp.com/biz/kanya-market-redding?...,64,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",4.0,"{'latitude': 40.564221, 'longitude': -122.353474}",[delivery],$,"{'address1': '970 Hartnell Ave', 'address2': '...",15302227609,(530) 222-7609,1299.516052
32,6V9a5Adj-aFy2ZSM09h0sg,boba-tea-and-coffee-redding,Boba Tea & Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/NTr5vM...,False,https://www.yelp.com/biz/boba-tea-and-coffee-r...,116,"[{'alias': 'taiwanese', 'title': 'Taiwanese'},...",4.0,"{'latitude': 40.5639496, 'longitude': -122.352...",[pickup],$,"{'address1': '2999 Churn Creek Rd', 'address2'...",15302211930,(530) 221-1930,1342.647391
33,8DQ7BdSsO5k6N5BE3Chhcw,new-china-redding,New China,https://s3-media2.fl.yelpcdn.com/bphoto/9l-4AY...,False,https://www.yelp.com/biz/new-china-redding?adj...,221,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,"{'latitude': 40.583114, 'longitude': -122.425491}","[delivery, pickup]",$$,"{'address1': '3669 Eureka Way', 'address2': ''...",15302469522,(530) 246-9522,5487.043285
34,4dzE57iooJCghpI5C34Ozg,fresh-fire-grill-redding,Fresh Fire Grill - Asian/American Fusion Resta...,https://s3-media3.fl.yelpcdn.com/bphoto/9twPO2...,False,https://www.yelp.com/biz/fresh-fire-grill-redd...,87,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 40.6115834469062, 'longitude': -1...","[delivery, pickup]",NaN,"{'address1': '161 Lake Blvd', 'address2': None...",15307685688,(530) 768-5688,4431.579792
35,0KYR7h1KXWhKKnNLTZohRQ,fresh-fire-grill-palo-cedro,Fresh Fire Grill,https://s3-media4.fl.yelpcdn.com/bphoto/oUGtRh...,False,https://www.yelp.com/biz/fresh-fire-grill-palo...,175,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 40.56439573760859, 'longitude': -...",[delivery],$,"{'address1': '9520 Deschutes Rd', 'address2': ...",15302271740,(530) 227-1740,10411.028811


## Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [87]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Redding_Thai-food.csv.gz', compression='gzip',index=False)